## Mimic EDA
#### Predicting and Understanding Unexpected Respiratory Decompensation in Critical Care Using Sparse and Heterogeneous Clinical Data

In [ ]:
import numpy as np
import pandas as pd
import psycopg2
import os 
import random
import datetime
from sqlalchemy import create_engine

random.seed(22891)

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

engine = create_engine("postgresql+psycopg2://{}:{}@/{}".format(sqluser, sqluser, dbname))

In [ ]:
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)

In [ ]:
from mimic_queries import services_query, demo_query, vent_query, vital_and_labs_query
from mimic_queries import clean_demo_df, comob_query

### Mimic Services

| Service |                                 Description                                               |
|---------|-------------------------------------------------------------------------------------------|
|   CMED  | Cardiac Medical - for non-surgical cardiac related admissions                             |
|   CSURG |	Cardiac Surgery - for surgical cardiac admissions                                         |
|   DENT  |	Dental - for dental/jaw related admissions                                                |
|   ENT   |	Ear, nose, and throat - conditions primarily affecting these areas                        |
|   GU	  | Genitourinary - reproductive organs/urinary system                                        |
|   GYN	  | Gynecological - female reproductive systems and breasts                                   |
|   MED	  | Medical - general service for internal medicine                                           |
|   NB	  | Newborn - infants born at the hospital                                                    |
|   NBB	  | Newborn baby - infants born at the hospital                                               |
|   NMED  |	Neurologic Medical - non-surgical, relating to the brain                                  |
|   NSURG |	Neurologic Surgical - surgical, relating to the brain                                     |
|   OBS	  | Obstetrics - conerned with childbirth and the care of women giving birth                  |
|   ORTHO |	Orthopaedic - surgical, relating to the musculoskeletal system                            |
|   OMED  |	Orthopaedic medicine - non-surgical, relating to musculoskeletal system                   |
|   PSURG |	Plastic - restortation/reconstruction of the human body (including cosmetic or aesthetic) |
|   PSYCH |	Psychiatric - mental disorders relating to mood, behaviour, cognition, or perceptions     |
|   SURG  |	Surgical - general surgical service not classified elsewhere                              |
|   TRAUM |	Trauma - injury or damage caused by physical harm from an external source                 |
|   TSURG |	Thoracic Surgical - surgery on the thorax, located between the neck and the abdomen       |
|   VSURG |	Vascular Surgical - surgery relating to the circulatory system                            |

In [ ]:
df_services = pd.read_sql(services_query(), conn)

In [ ]:
df_services.head(3)

In [ ]:
df_services.curr_service.value_counts()

### Mimic Demographics

In [ ]:
df_demo = pd.read_sql_query(demo_query(), conn)

In [ ]:
df_demo_clean = clean_demo_df(df_demo, 27)  #  stay in ICU a minimum of 27h

In [ ]:
len(df_demo), len(df_demo_clean)

In [ ]:
df_demo_clean.adult_icu.value_counts()

In [ ]:
df_demo_clean.age.value_counts()

In [ ]:
df_demo_clean.admission_type.value_counts()

In [ ]:
df_demo_clean.ethnicity.value_counts()

In [ ]:
df_demo_clean.diagnosis.value_counts()

In [ ]:
df_demo_clean.first_hosp_stay.value_counts()

In [ ]:
df_demo_clean.first_icu_stay.value_counts()

In [ ]:
df_demo_clean.mort_icu.value_counts()

### Mimic Mechanical Ventilation

In [ ]:
df_vent = pd.read_sql_query(vent_query(), conn)
# there are 34 instances of MV without an icustay_id, we remove those
# also, there are a few indididuals with multiple instances of MV that we remove
df_vent = df_vent[~df_vent.icustay_id.isnull()]
df_vent = df_vent.astype({"icustay_id": int})
df_vent = df_vent[df_vent['ventnum'] == 1]

In [ ]:
len(df_vent), len(df_demo_clean)

In [ ]:
df_cohort = df_demo_clean.merge(df_vent, left_on='icustay_id', right_on='icustay_id', how='left')
df_cohort = df_cohort.merge(df_services, 
                            left_on=['subject_id','hadm_id'], 
                            right_on=['subject_id','hadm_id'], how='inner')


df_cohort = df_cohort[(df_cohort.transfertime > df_cohort.admittime) & 
                      (df_cohort.transfertime < df_cohort.outtime)]

df_cohort = df_cohort[(np.isnan(df_cohort.ventnum)) | 
                      (df_cohort.mv_start > df_cohort.transfertime)]

In [ ]:
len(df_demo_clean), len(df_cohort)

In [ ]:
df_demo_clean.subject_id.nunique(), df_cohort.subject_id.nunique()

#### Almost exactly following:
Cohort: We used data from adult patients from MIMIC
III in this study (age greater than 16 at admission). Next,
for patients with multiple hospital or ICU stays, we only
considered the first ICU stay for the first hospital stay. We
excluded patients admitted under surgical service because
surgical patients are frequently intubated due to anesthesia
rather than respiratory failure. Patients who were intubated on
admission were excluded, as were patients who were intubated
or discharged from the ICU within 27 hours from admission.
Finally, we removed patients who requested a withdrawal of
care within the first 27 hours as these patients will not be
intubated despite respiratory decompensation

In [ ]:
# keep only adult patient admissions
df_cohort = df_cohort.loc[df_cohort.adult_icu == 1]
# keep only admissions that are first time ICU admissions and hospitalizations
df_cohort = df_cohort.loc[(df_cohort.first_icu_stay == 1) & (df_cohort.first_hosp_stay == 1)]
# remove admissions of patients intubated at admition time
df_cohort = df_cohort[np.isnan(df_cohort.ventnum) | (df_cohort.mv_start - df_cohort.admittime).dt.days > 0]
# remove admissions of surgical patients 
surgical = ['CSURG', 'NSURG', 'ORTHO', 'SURG', 'TSURG', 'VSURG']
df_cohort = df_cohort.loc[~np.isin(df_cohort.curr_service, surgical)]

In [ ]:
len(df_cohort), df_cohort.subject_id.nunique()

In [ ]:
cols = list(set(df_cohort.columns) - {'transfertime', 'prev_service', 'curr_service'})
df_cohort = df_cohort.sort_values(['transfertime']).drop_duplicates(cols, keep='last')

In [ ]:
len(df_cohort)

In [ ]:
df_cohort.ventnum.value_counts(dropna=False)

In [ ]:
df_cohort.head(3)

In [ ]:
conn.execute("DROP TABLE IF EXISTS mv_users;")

In [ ]:
conn.close()
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)
df_columns = ['subject_id', 'hadm_id', 'icustay_id', 'ventnum', 'mv_start', 'transfertime']
df_cohort[df_columns].to_sql('mv_users', conn, if_exists='replace', index=False)

In [ ]:
conn.close()
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)

In [ ]:
df_comob = pd.read_sql(comob_query(), conn)

In [ ]:
df_comob.head()

#### Loosely following:
Data extraction: For each patient in our cohort, we
extracted data from a window of size W hours, located L
hours before a given event time. Specifically, we extracted
a number of features, (see Table I), from the data window
[te − W − L, te − L], where te represents the event time. For
intubated patients, the event time was the time of intubation.
For non-intubated patients, the event time was a random time
after the 27th hour during their ICU stay. For non-intubated
patients with a code status change (e.g. changed to do not
resuscitate or do not intubate), we ensured that the event
time was before the code status change. For all patients,
we evaluated and extracted features from data windows with
window sizes of W ∈ {8, 12, 16, 20, 24} hours and lead times
of L ∈ {3, 6, 9, 12, 15} hours.

In [ ]:
vitals_labs_df = pd.read_sql(vital_and_labs_query(27, 3), conn)

In [ ]:
len(vitals_labs_df), vitals_labs_df.subject_id.nunique()

In [ ]:
vitals_labs_df.columns

In [ ]:
from IPython.display import display
pd.options.display.max_columns = None
display(vitals_labs_df.head())

In [ ]:
conn.close()

In [ ]:
print(vital_and_labs_query(27, 3))